In [1]:
import numpy as np
import pandas
from matplotlib import pyplot as plt

#add taxes foncieres and income tax (ish) and insurance

class ListingObject:
    
    def __init__(self
            , link='https://www.pap.fr/'
            , surface=57
            , price=485000
            , cost=3920/12
            , rent=1000
            , notary=0.08
            , agency=0
            , yearsOfDebt=15
            , inflation=0.02
            , yearlyInterest=0.019
            , taxeFonciere=0
            , contribution=200000
            , contributionBis=50000
            ):
        self.link=link
        self.surface=surface
        self.price=price
        self.rent=rent
        self.taxeFonciere=taxeFonciere
        self.impot=0.3
        self.cost=cost
        self.notary=notary
        self.agency=agency
        self.yearsOfDebt=yearsOfDebt
        self.inflation=inflation
        self.monthlyInflation=pow(1+inflation, 1/12) - 1
        self.yearlyInterest=yearlyInterest
        self.contribution=contribution
        self.debtRequired=price*(1+agency+notary)-contribution
        self.monthlyInterestRate=pow(1+yearlyInterest, 1/12) - 1
        self.monthlyPayments=self.debtRequired*self.monthlyInterestRate/(1-pow(1+self.monthlyInterestRate, -(12*self.yearsOfDebt)))
        self.costOfCost=self.cost/(self.monthlyInterestRate/(1-pow(1+self.monthlyInterestRate, -(12*self.yearsOfDebt))))
        #build timeline view
        self.rowLines=[]
        self.debtLeft=self.debtRequired
        self.cumCash=0
        self.cumOwned=self.contribution
        self.cumInterest=0
        self.cumCost=self.price*(self.agency+self.notary)
        self.contributionBis=contributionBis
        self.debtShareBis=(self.contribution/2 - self.contributionBis + self.debtRequired/2)/self.debtRequired
        for x in range(12*self.yearsOfDebt):
            self.interestPaid = self.monthlyInterestRate*self.debtLeft
            self.totalTax = max(self.rent*0.25 - self.interestPaid - self.cost + self.taxeFonciere/12, self.taxeFonciere/12)
            self.paidIn = self.monthlyPayments-self.interestPaid
            self.debtLeft-=self.paidIn
            self.cumOwned+=self.paidIn
            self.cumCash+=self.paidIn
            self.cumInterest+=self.interestPaid
            self.cumCost+=self.cost
            self.rowLines.append({'cashIn':self.rent
                                  , 'cashOut':self.monthlyPayments+self.cost
                                  , 'interestPaid':self.interestPaid
                                  , 'invested':self.paidIn
                                  , 'debt':self.debtLeft
                                  , 'sumCashed': self.cumCash/(self.debtRequired+self.contribution)
                                  , 'sumInterest':self.cumInterest/(self.debtRequired+self.contribution)
                                  , 'sumOwned':self.cumOwned
                                  , 'sumCost':self.cumCost/(self.debtRequired+self.contribution)
                                  , 'sumRev':self.rent*x
                                  , 'cost':self.cost
                                  , 'period': x
                                  , 'tax': self.totalTax
                                 })
        self.data=pandas.DataFrame(self.rowLines)
        self.data['npvOne']= - self.data['cost']/2 -self.data['tax']/2 - self.monthlyPayments*(1-self.debtShareBis) + self.data['cashIn']/2 #/((1+self.monthlyInflation)**self.data['period']))
        self.data['npvBis']= - self.data['cost']/2 -self.data['tax']/2 - self.monthlyPayments*(self.debtShareBis) + self.data['cashIn']/2  #/((1+self.monthlyInflation)**self.data['period']))
        self.NPVCash =  - self.data['cost'] - (self.monthlyPayments/((1+self.monthlyInflation)**self.data['period'])) #-self.data['tax']
        
    def analyze(self):
        print("Total Debt & Interest & Notary & cost: {:4.2f} & {:4.2f} & {:4.2f} & {:4.2f}"
              .format(self.debtRequired, self.data['interestPaid'].sum(), self.notary*self.price, self.costOfCost))
        print("Total vs monthly cash balance, monthly dues & cost: {:4.2f} & {:4.2f} & {:4.2f} & {:4.2f}"
              .format(-self.data['cashOut'].sum()+self.data['cashIn'].sum(), -self.monthlyPayments-self.cost+self.rent, self.monthlyPayments, self.cost))
        print("Purchase vs rental per m2: {:4.2f} & {:4.2f} & {:4.2f}"
              .format(self.price/self.surface, self.rent/self.surface, self.rent))
        print("Share of cash L & V: {:4.2f} & {:4.2f}"
              .format((1-self.debtShareBis) * self.monthlyPayments - self.rent/2 +self.cost/2 + self.taxeFonciere/24, ((self.debtShareBis) * self.monthlyPayments - self.rent/2 +self.cost/2 + self.taxeFonciere/24)) )
        print("Return expected: {:4.2f} & {:4.2f} & {:4.2f}"
              .format(-self.data['interestPaid'].sum() - self.data['cost'].sum() +self.data['cashIn'].sum()-self.taxeFonciere*self.yearsOfDebt
                      , (-self.data['interestPaid'].sum() - self.data['cost'].sum() +self.data['cashIn'].sum()-self.taxeFonciere*self.yearsOfDebt)/(self.debtRequired + self.contribution) 
                     , ((self.rent-self.cost)*1200 + self.taxeFonciere)/(self.debtRequired +self.contribution))
        )
        
    def graph(self):
        ax1 = self.data[['sumCashed', 'sumInterest', 'sumCost']].plot(kind='area', stacked=True)
        ax2 = ax1.twinx()
        ax1.set_ylim(0,1.1)
        ax2.spines['right'].set_position(('axes', 1.0))
        ax2.set_ylim(0,(self.debtRequired+self.contribution)*1.1)
        ax2 = self.data[['sumOwned', 'sumRev']].plot(kind='line', ax=ax2, colormap='plasma')
        ax2.legend(loc="upper right")
        ax3 = self.data[['interestPaid', 'invested', 'cost', 'tax']].plot(kind='area', stacked=True)
        self.data['cashIn'].plot(ax=ax3,colormap='plasma')
        self.data[['npvOne', 'npvBis']].plot(kind='area', stacked=False)
        
    def valuate(self):
        NPV = self.price - self.contribution - self.price*(self.agency+self.notary) + self.NPVCash.sum() + self.data['cashIn'].sum()
        print(NPV)
        #print("Share of cash L & V:")
        print(pandas.concat([self.debtShareBis * self.NPVCash + self.data['cashIn']/2, (1-self.debtShareBis) * self.NPVCash + self.data['cashIn']/2], axis=1, sort=False))
        print(((1-self.debtShareBis) * self.NPVCash + self.data['cashIn']/2).where(lambda x : x>1).dropna().count())

In [ ]:
C=ListingObject(
                 link='ee'
            , surface=40.4
            , price=320000
            , cost=180
            , rent=1000
            , notary=24100/320000
            , agency=10750/320000
            , yearsOfDebt=10
            , inflation=0.015
            , yearlyInterest=0.014
            , contribution=220000
            , contributionBis=60000
            , taxeFonciere = 740)
C.analyze()
C.graph()